<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/Exam_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head(1)

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,0.55,0.4125,0.1625,1.715145,0.609514,0.396893,0.56699,4.0


In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')

enc.fit_transform(df[['Sex']])


df['Sex'] = enc.transform(df[['Sex']]).toarray()

In [ ]:
X = df.drop(columns=['Age', 'id'], axis=1)
y = df['Age']

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_train = poly.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_poly_train, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 3.950846393721845


In [ ]:
X = df.drop(columns=['Age', 'id', 'Sex'], axis=1)
y = df['Age']

model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [ ]:
test = pd.read_csv('test.csv')
y_test_pred = model.predict(test.drop(columns=['id', 'Sex'], axis=1))
submission_df = pd.DataFrame({'id': test['id'], 'Age': y_test_pred})
submission_df.to_csv('sample_submission.csv',
                     index=False)

In [ ]:
submission_df

,id,Age
0,15000,6.818378
1,15001,8.734077
2,15002,5.588068
3,15003,7.782630
4,15004,6.961167
...,...,...
9995,24995,9.263205
9996,24996,8.572384
9997,24997,9.885525
9998,24998,8.546089


In [ ]:
import pandas as pd
from sklearn.linear_model import HuberRegressor, RANSACRegressor, TheilSenRegressor, Ridge, Lasso
from sklearn.feature_selection import RFECV
from sklearn.ensemble import StackingRegressor, VotingRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error


train_data = pd.read_csv('train.csv')

X = train_data.drop(columns=['id', 'Age'])
y = train_data['Age']

X_cleaned = X[X['Sex'].isin(['I', 'F', 'M'])]
y_cleaned = y[X['Sex'].isin(['I', 'F', 'M'])]

categorical_features = ['Sex']
numeric_features = X_cleaned.columns.difference(categorical_features)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])


X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)


base_models = [
    ('huber', HuberRegressor()),
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor())
]

stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=Ridge())


rfecv = RFECV(estimator=Ridge(), step=1, scoring='neg_mean_squared_error')
rfecv_stacking_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rfecv', rfecv),
    ('stacking', stacking_regressor)
])

param_distributions = {
    'stacking__final_estimator__alpha': [0.1, 1.0, 10.0],
    'stacking__huber__epsilon': [1.35, 1.5, 1.75],
}

random_search = RandomizedSearchCV(rfecv_stacking_pipeline, param_distributions, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)


random_search.fit(X_train_cleaned, y_train_cleaned)

best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_cleaned)
final_mse = mean_squared_error(y_test_cleaned, y_pred)

final_mse, random_search.best_params_


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


(3.8990755754925073,
 {'stacking__huber__epsilon': 1.75, 'stacking__final_estimator__alpha': 1.0})

In [ ]:
import pandas as pd
from sklearn.linear_model import HuberRegressor, RANSACRegressor, TheilSenRegressor, Ridge
from sklearn.feature_selection import RFECV
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Load the data
train_data = pd.read_csv('train.csv')

# Separate features and target variable
X = train_data.drop(columns=['id', 'Age'])
y = train_data['Age']

# Clean the data by filtering out invalid categorical entries
X_cleaned = X[X['Sex'].isin(['I', 'F', 'M'])]
y_cleaned = y[X['Sex'].isin(['I', 'F', 'M'])]

# Define categorical and numerical features
categorical_features = ['Sex']
numeric_features = X_cleaned.columns.difference(categorical_features)

# Preprocessing pipeline with PolynomialFeatures, OneHotEncoder, and StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Split the data into training and testing sets
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Define base models for stacking
base_models = [
    ('huber', HuberRegressor()),
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor())
]

# Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=Ridge())

# Pipeline with RFECV, Stacking Regressor, and Preprocessing
rfecv = RFECV(estimator=Ridge(), step=1, scoring='neg_mean_squared_error')
rfecv_stacking_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rfecv', rfecv),
    ('stacking', stacking_regressor)
])

# Define the parameter distribution for RandomizedSearchCV
param_distributions = {
    'stacking__final_estimator__alpha': [0.1, 1.0, 10.0],
    'stacking__huber__epsilon': [1.35, 1.5, 1.75],
}

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(rfecv_stacking_pipeline, param_distributions, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)

# Fit the model
random_search.fit(X_train_cleaned, y_train_cleaned)

# Evaluate the best model
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_cleaned)
final_mse = mean_squared_error(y_test_cleaned, y_pred)

# Output the results
final_mse, random_search.best_params_


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res,

(3.675462903468458,
 {'stacking__huber__epsilon': 1.75, 'stacking__final_estimator__alpha': 1.0})

In [ ]:
test = pd.read_csv('test.csv')
y_test_pred = random_search.predict(test.drop(columns=['id'], axis=1))
submission_df = pd.DataFrame({'id': test['id'], 'Age': y_test_pred})
submission_df.to_csv('sample_submission.csv',
                     index=False)

In [ ]:
import pandas as pd
from sklearn.linear_model import HuberRegressor, RANSACRegressor, TheilSenRegressor, Ridge
from sklearn.feature_selection import RFECV
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Load the data
train_data = pd.read_csv('train.csv')

# Separate features and target variable
X = train_data.drop(columns=['id', 'Age'])
y = train_data['Age']

# Clean the data by filtering out invalid categorical entries
X_cleaned = X[X['Sex'].isin(['I', 'F', 'M'])]
y_cleaned = y[X['Sex'].isin(['I', 'F', 'M'])]

# Define categorical and numerical features
categorical_features = ['Sex']
numeric_features = X_cleaned.columns.difference(categorical_features)

# Preprocessing pipeline with PolynomialFeatures, OneHotEncoder, and StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Split the data into training and testing sets
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Define base models for stacking
base_models = [
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor())
]

# Stacking Regressor with HuberRegressor as the final estimator
stacking_regressor = StackingRegressor(estimators=base_models,
                                       final_estimator=HuberRegressor(epsilon=1.75, alpha=1.0))

# Pipeline with RFECV, Stacking Regressor, and Preprocessing
rfecv = RFECV(estimator=Ridge(), step=1, scoring='neg_mean_squared_error')
rfecv_stacking_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rfecv', rfecv),
    ('stacking', stacking_regressor)
])

# Define the parameter grid for GridSearchCV
param_grid = {
    'stacking__final_estimator__epsilon': [1.7, 1.75, 1.8],
    'stacking__final_estimator__alpha': [0.1, 1.0, 10.0]
}

# Perform GridSearchCV
grid_search = GridSearchCV(rfecv_stacking_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train_cleaned, y_train_cleaned)

# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_cleaned)
final_mse = mean_squared_error(y_test_cleaned, y_pred)

# Output the results
final_mse, grid_search.best_params_


(4.2709359368715925,
 {'stacking__final_estimator__alpha': 0.1,
  'stacking__final_estimator__epsilon': 1.8})

In [ ]:
import pandas as pd
from sklearn.linear_model import HuberRegressor, RANSACRegressor, TheilSenRegressor, Ridge
from sklearn.feature_selection import RFECV
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Load the data
train_data = pd.read_csv('train.csv')

# Separate features and target variable
X = train_data.drop(columns=['id', 'Age'])
y = train_data['Age']


X_cleaned = X[X['Sex'].isin(['I', 'F', 'M'])]
y_cleaned = y[X['Sex'].isin(['I', 'F', 'M'])]


categorical_features = ['Sex']
numeric_features = X_cleaned.columns.difference(categorical_features)


preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Split the data into training and testing sets
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Define base models for stacking
base_models = [
    ('huber', HuberRegressor()),
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor())
]

# Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models,
                                      #  final_estimator=Ridge()
                                      final_estimator=RandomForestRegressor(n_estimators=10,
                                      random_state=42)
                                       )

# Pipeline with RFECV, Stacking Regressor, and Preprocessing
rfecv = RFECV(estimator=Ridge(), step=1, scoring='neg_mean_squared_error')
rfecv_stacking_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rfecv', rfecv),
    ('stacking', stacking_regressor)
])

# Define the parameter distribution for RandomizedSearchCV
param_distributions = {
    'stacking__final_estimator__alpha': [0.1, 1.0, 10.0],
    'stacking__huber__epsilon': [1.35, 1.5, 1.75],
}

# Perform RandomizedSearchCV
# random_search = RandomizedSearchCV(rfecv_stacking_pipeline, param_distributions, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)

# # Fit the model
# random_search.fit(X_train_cleaned, y_train_cleaned)

# # Evaluate the best model
# best_model = random_search.best_estimator_
# y_pred = best_model.predict(X_test_cleaned)
# final_mse = mean_squared_error(y_test_cleaned, y_pred)

# # Output the results
# final_mse, random_search.best_params_

In [3]:
import pandas as pd
from sklearn.linear_model import HuberRegressor, RANSACRegressor, TheilSenRegressor, Ridge
from sklearn.feature_selection import RFECV
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error

train_data = pd.read_csv('train.csv')

# Separate features and target variable
X = train_data.drop(columns=['id', 'Age'])
y = train_data['Age']

# Clean the data by filtering out invalid categorical entries
X_cleaned = X[X['Sex'].isin(['I', 'F', 'M'])]
y_cleaned = y[X['Sex'].isin(['I', 'F', 'M'])]

# Define categorical and numerical features
categorical_features = ['Sex']
numeric_features = X_cleaned.columns.difference(categorical_features)

# Preprocessing pipeline with PolynomialFeatures, OneHotEncoder, and StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Split the data into training and testing sets
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Define base models for stacking
base_models = [
    ('huber', HuberRegressor()),
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor())
]

# Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models,
                                      #  final_estimator=Ridge()
                                      final_estimator=RandomForestRegressor(n_estimators=10,
                                      random_state=42)
                                       )

# Pipeline with RFECV, Stacking Regressor, and Preprocessing
rfecv = RFECV(estimator=Ridge(), step=1, scoring='neg_mean_squared_error')
rfecv_stacking_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rfecv', rfecv),
    ('stacking', stacking_regressor)
])


y_pred = rfecv_stacking_pipeline.predict(X_test_cleaned)
final_mse = mean_squared_error(y_test_cleaned, y_pred)

# Output the results
final_mse

NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [55]:
import pandas as pd
from sklearn.linear_model import HuberRegressor, RANSACRegressor, TheilSenRegressor, Ridge, ElasticNet, Lasso
from sklearn.feature_selection import RFECV
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Load the data
train_data = pd.read_csv('train.csv')

# Separate features and target variable
X = train_data.drop(columns=['id', 'Age'])
y = train_data['Age']

# Clean the data by filtering out invalid categorical entries
X_cleaned = X[X['Sex'].isin(['I', 'F', 'M'])]
y_cleaned = y[X['Sex'].isin(['I', 'F', 'M'])]

# Define categorical and numerical features
categorical_features = ['Sex']
numeric_features = X_cleaned.columns.difference(categorical_features)

# Preprocessing pipeline with PolynomialFeatures, OneHotEncoder, and StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            # ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Split the data into training and testing sets
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Define base models for stacking
base_models = [
    ('huber', HuberRegressor()),
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor()),
    ('lasso', Lasso()),
    ('elastic', ElasticNet()),
    ('ridge', Ridge())

]

# Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models,
                                       final_estimator=RandomForestRegressor(n_estimators=10,
                                       random_state=42))
voting_regressor = VotingRegressor(estimators=[
    ('huber', HuberRegressor()),
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor()),
    # ('lasso', Lasso()),
    # ('elastic', ElasticNet()),
    # ('ridge', Ridge()),
    ('stacking', stacking_regressor)
])

# Pipeline with RFECV, Stacking Regressor, and Preprocessing
rfecv = RFECV(estimator=Ridge(), step=1, scoring='neg_mean_squared_error')
rfecv_stacking_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rfecv', rfecv),
    ('stacking', voting_regressor)
])

rfecv_stacking_pipeline.fit(X_train, y_train)

y_pred = rfecv_stacking_pipeline.predict(X_test)

final_mse = mean_squared_error(y_test, y_pred)
final_mse


4.148559732951034

In [12]:
data = pd.read_csv('train.csv')

# Define the feature columns and the target column
features = data.drop(['Age', "id"], axis=1)
target = data['Age']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Define the OneHotEncoder for the 'Sex' column
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Sex'])
    ],
    remainder='passthrough'
)

# Define the HuberRegressor model
model = Pipeline(steps=[
    ('preprocessor', column_transformer),
    ('regressor', HuberRegressor()),
    # ('scaler', StandardScaler())
])

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 4.215701516343436


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [17]:
data = pd.read_csv('train.csv')

# Define the feature columns and the target column
features = data.drop('Age', axis=1)
target = data['Age']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Define the OneHotEncoder for the 'Sex' column and apply StandardScaler after encoding
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Sex'])
    ],
    remainder='passthrough'
)

# Define the full pipeline with StandardScaler and HuberRegressor
model = Pipeline(steps=[
    ('preprocessor', column_transformer),
    ('scaler', StandardScaler()),  # Scale the data after one-hot encoding
    ('regressor', HuberRegressor())
])

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 4.215531839052196


In [19]:
data = pd.read_csv('train.csv')

# Define the feature columns and the target column
features = data.drop('Age', axis=1)
target = data['Age']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Define the OneHotEncoder for the 'Sex' column and apply StandardScaler after encoding
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Sex'])
    ],
    remainder='passthrough'
)

# Define the full pipeline with StandardScaler and HuberRegressor
model = Pipeline(steps=[
    ('preprocessor', column_transformer),
    ('scaler', StandardScaler()),  # Scale the data after one-hot encoding
    ('regressor', TheilSenRegressor())
])

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 4.163335487768201


In [53]:
features = data.drop('Age', axis=1)
target = data['Age']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Sex'])
    ],
    remainder='passthrough'
)

base_models = [
    ('huber', HuberRegressor()),
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor()),
    ('lasso', Lasso()),
    ('elastic', ElasticNet()),
    ('ridge', Ridge()),
    ('linea', LinearRegression())

]

# Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models,
                                       final_estimator=LinearRegression())
voting_regressor = VotingRegressor(estimators=[
    ('huber', HuberRegressor()),
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor()),
    ('linea', LinearRegression()),
    ('stacking', stacking_regressor)
])

# Pipeline with RFECV, Stacking Regressor, and Preprocessing
# rfecv = RFECV(estimator=Ridge(), step=1, scoring='neg_mean_squared_error')
rfecv_stacking_pipeline = Pipeline([
    ('preprocessor', column_transformer),
    # ('rfecv', rfecv),
    ('stacking', voting_regressor)
])

rfecv_stacking_pipeline.fit(X_train, y_train)

y_pred = rfecv_stacking_pipeline.predict(X_test)

final_mse = mean_squared_error(y_test, y_pred)
final_mse

4.146164723675939

In [60]:
data.columns

Index(['id', 'Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight', 'Age'],
      dtype='object')

In [ ]:
columns = ['id', 'Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight']

num_columns = ['id','Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight']

features = data[columns]
target = data['Age']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Sex']),
        ('num', Pipeline(steps=[
            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('scaler', StandardScaler())
        ]), num_columns),
    ],
    remainder='passthrough'
)

base_models = [
    ('huber', HuberRegressor()),
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor()),
    ('lasso', Lasso()),
    ('elastic', ElasticNet()),
    ('ridge', Ridge()),
    ('linea', LinearRegression())

]

# Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models,
                                       final_estimator=LinearRegression())
voting_regressor = VotingRegressor(estimators=[
    ('huber', HuberRegressor()),
    ('ransac', RANSACRegressor()),
    ('theilsen', TheilSenRegressor()),
    ('linea', LinearRegression()),
    ('stacking', stacking_regressor)
])

# Pipeline with RFECV, Stacking Regressor, and Preprocessing
rfecv = RFECV(estimator=LinearRegression(), step=1, scoring='neg_mean_squared_error')
rfecv_stacking_pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('rfecv', rfecv),
    ('stacking', voting_regressor)
])

rfecv_stacking_pipeline.fit(X_train, y_train)

y_pred = rfecv_stacking_pipeline.predict(X_test)

final_mse = mean_squared_error(y_test, y_pred)
final_mse